# Notebook 06 - Lot Area and Frontage data cleaning and fixing

## Objectives
* Clean data
* Evaluate and process missing data
* Fix potential issues with data in feature (LotArea, LotFrontage)

## Inputs
* inputs/datasets/cleaning/kitchen.csv

## Outputs
* Clean and fix (missing and potentially wrong) data in given column
* After cleaning is completed, we will save current dataset in inputs/datasets/cleaning/lot_features.csv

## Change working directory
In This section we will get location of current directory and move one step up, to parent folder, so App will be accessing project folder.

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chdir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("you have set a new current directory")

Confirm new current directory

In [3]:
current_dir = os.getcwd()
current_dir

We need to check current working directory

In [4]:
current_dir

We can see that current is **jupyter_notebooks**, as current notebook is in subfolder. We will go one step up to parent directory, what will be our project main directory.
Print out to confirm working directory

In [5]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()
current_dir

## Loading Dataset

In [6]:
import pandas as pd

df = pd.read_csv("inputs/datasets/cleaning/kitchen.csv")
df.head()

## Exploring Data

We will get all features that are missing data as a list

In [7]:
df['LotArea'].isnull().sum()

In [8]:
df['LotFrontage'].isna().sum()

We can see that there is no missing values in LotArea, but LotFrontage - has 259

As it is Linear feet of street connected to property, it is very hard to decide how it could be correlated to other features, as it does not depend on house owner, but on designer. 

We will replace all missing values with mean

In [9]:
df.loc[:, 'LotFrontage'] = df['LotFrontage'].fillna(value=df['LotFrontage'].mean())

We will check if LotFrontage and LotArea data types

In [10]:
# Print the data type of 'LotFrontage'
print("Data type of 'LotFrontage':", df['LotFrontage'].dtype)

# Print the data type of 'LotArea'
print("Data type of 'LotArea':", df['LotArea'].dtype)

LotFrontage need converting to integer

In [11]:
df['LotFrontage'] = df['LotFrontage'].round().astype(int)

Now all we can do is check if LotArea values are correct:

Lot size generally should be bigger than:
* WoodDeckSF - Wood deck area in square feet
* GarageArea - Size of garage in square feet
* GrLivArea - above ground living area in square feet
* EnclosedPorch - Enclosed porch area in square feet
* OpenPorchSF - Open porch area in square feet

So we have to summ all those features values, and they have to be smaller than lot area.
You can not build a house that has features on too small plot area

In [12]:
# Summing relevant area features
df['SummedAreas'] = df['WoodDeckSF'] + df['GarageArea'] + df['GrLivArea'] + \
                    df['EnclosedPorch'] + df['OpenPorchSF']

# Creating a boolean mask where summed areas exceed LotArea
df['IsAreaExceeded'] = df['SummedAreas'] > df['LotArea']

# Filtering rows where the built-up area exceeds the lot of area
inconsistent_records = df[df['IsAreaExceeded']]

# Display these records
inconsistent_records

We can see that all  LotArea values are within range.

## Removing added columns

We will use same code as in previous cleaning notebook 04_basement.ipynb

In [13]:
# Removing Extra columns that originally do not belong to dataset, as we have created them

df_original_features = pd.read_csv("outputs/datasets/collection/HousePricesRecords.csv")

# Identify columns in df that are also in df_original
common_columns = df.columns.intersection(df_original_features.columns)

# Filter df to only include those common columns
df = df[common_columns]

df

## Saving current dataset

We will save current dataset as inputs/datasets/cleaning/lot_features.csv

In [14]:
df.to_csv('inputs/datasets/cleaning/lot_features.csv', index=False)

## Next step is cleaning Masonry Veneer and Porch features- cleaning and fixing data in garages